In [1]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer\\research'

In [2]:
import os
from pathlib import Path

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer'

In [5]:
from dataclasses import dataclass

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_train_split: float
    params_validation_split: float
    params_test_split: float
    

In [6]:
from farm_copilot.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from farm_copilot.utils.common import read_yaml, create_directories
from tensorflow import keras
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH ) -> None:
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepared_base_model = self.config.prepare_base_model
        params = self.params
        training_data = Path(self.config.data_ingestion.local_data_file)

        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepared_base_model.updated_base_model_path),
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_train_split=params.TRAIN_SPLIT,
            params_test_split=params.TEST_SPLIT,
            params_validation_split=params.VALIDATION_SPLIT
        )

        return training_config

In [8]:
class Training:
    def __init__(self, config: TrainingConfig) -> None:
        self.config = config
    
    
    def get_base_model(self):
        self.model = keras.models.load_model(filepath=self.config.updated_base_model_path)

    
    def load_dataset(self):
        self.dataset = keras.preprocessing.image_dataset_from_directory(
                        directory=self.config.training_data,
                        shuffle=True,
                        label_mode='categorical',
                        image_size = tuple(self.config.params_image_size), 
                        batch_size = self.config.params_batch_size,
                        )
    
    
    def train_validation_test_split(self, shuffle: bool = True, seed: int = 49, shuffle_size: int = 1000):
        # Get the len OF the ds
        ds_size = len(self.dataset)

        # Shuffle datset
        if shuffle:
            self.dataset = self.dataset.shuffle(shuffle_size, seed=49)

        # Compute the ds split sizes
        train_size = int(self.config.params_train_split * ds_size)
        val_size = int(self.config.params_validation_split * ds_size)

        # Get the dataset partitions
        self.train_ds = self.dataset.take(train_size)
        self.val_ds = self.dataset.skip(train_size).take(val_size)
        self.test_ds = self.dataset.skip(train_size).skip(val_size)


    def data_augumentation(self):
        augmentation_layers = keras.Sequential([
            keras.layers.RandomFlip('horizontal_and_vertical'),
            keras.layers.RandomRotation(0.2),
            keras.layers.RandomZoom(0.2),            
            keras.layers.RandomContrast(0.2)
        ])
        
        # Augument the training set
        self.train_ds = self.train_ds.map(lambda x, y: (augmentation_layers(x), y))


    def prefetch(self):
        # Preload images into memory using combined CPU and GPU
        self.train_ds = self.train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
        self.val_ds = self.val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


    def train(self):        
        self.model.fit(
            self.train_ds,
            epochs=self.config.params_epochs,            
            validation_data=self.val_ds
        )


        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

        
    @staticmethod
    def save_model(path: Path, model: keras.Model):
        keras.models.save_model(model=model, filepath=path)


In [9]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.load_dataset()
    training.train_validation_test_split()    
    #training.data_augumentation()
    training.prefetch()
    training.train()
except Exception as e:
    raise e

[2024-04-04 19:20:47,575: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-04 19:20:47,599: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-04 19:20:47,602: INFO: common: created directory at: artifacts]
[2024-04-04 19:20:47,604: INFO: common: created directory at: artifacts\training]


KeyboardInterrupt: 